# [3주차 심화과제] GPT로 뉴스 기사 분류 모델 학습하기


In [1]:
import torch
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader

tokenizer = torch.hub.load(
    "huggingface/transformers",  # huggingface/pytorch-transformers 에서 huggingface/transformers 로 변경됨
    "tokenizer",  # github.com/huggingface/transformers repo의 root에 'hubconf.py' 파일의 'tokenizer' 함수를 가져옴
    "openai-gpt",  # tokenizer 함수의 인자로 전달됨
    trust_repo=True,  # huggingface/transformers 를 믿을 수 있는 repo로 설정 -> 로컬의 trusted_list에 추가됨
    clean_up_tokenization_spaces=True,  # 현재는 True가 default 이지만 향후 False가 default로 바뀔예정 : 문장 중간에 나오는 띄어쓰기와 관련된 이슈 > https://github.com/huggingface/transformers/issues/31884
)

tokenizer.pad_token = tokenizer.unk_token

Using cache found in /Users/obov/.cache/torch/hub/huggingface_transformers_main


In [2]:
ds = load_dataset("fancyzhx/ag_news")

ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [3]:
tokenizer(
    ds["train"][0]["text"],
    padding="max_length",
    truncation=False,
    max_length=400,
).attention_mask

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [4]:
unique_labels = set(ds["train"]["label"])
count_of_label = len(unique_labels)
count_of_label

4

In [5]:
label_names = ds["train"].features["label"].names

label_names

['World', 'Sports', 'Business', 'Sci/Tech']

In [6]:
def collate_fn(batch):
    max_len = 400
    texts, labels = [], []
    for row in batch:
        labels.append(row["label"])
        texts.append(row["text"])

    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=False,
        max_length=max_len,
    )

    texts = torch.LongTensor(tokenized.input_ids)
    attention_mask = torch.LongTensor(tokenized.attention_mask)
    labels = torch.LongTensor(labels)

    return texts, attention_mask, labels


train_loader = DataLoader(
    ds["train"], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds["test"], batch_size=64, shuffle=False, collate_fn=collate_fn
)

In [7]:
# 1200개 샘플로 줄이기
small_ds = DatasetDict(
    {
        "train": ds["train"].select(range(1200)),
        "test": ds["test"].select(range(1200)),
    }
)
print(small_ds)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [8]:
small_train_loader = DataLoader(
    small_ds["train"], batch_size=64, shuffle=True, collate_fn=collate_fn
)
small_test_loader = DataLoader(
    small_ds["test"], batch_size=64, shuffle=False, collate_fn=collate_fn
)

In [9]:
model = torch.hub.load(
    "huggingface/transformers",  # huggingface/pytorch-transformers 에서 huggingface/transformers 로 변경됨
    "model",  # 'model' 함수를 가져옴
    "openai-gpt",
)
model

Using cache found in /Users/obov/.cache/torch/hub/huggingface_transformers_main


OpenAIGPTModel(
  (tokens_embed): Embedding(40478, 768)
  (positions_embed): Embedding(512, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x Block(
      (attn): Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
)

In [10]:
from torch import nn


class TextClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = torch.hub.load(
            "huggingface/transformers",  # huggingface/pytorch-transformers 에서 huggingface/transformers 로 변경됨
            "model",  # 'model' 함수를 가져옴
            "openai-gpt",
        )

        self.classifier = nn.Linear(768, 4)

    def forward(self, x, attention_mask):
        x = self.encoder(x, attention_mask)["last_hidden_state"]
        x = x.max(dim=1)[0]  # 가장 큰 값만 선택
        x = self.classifier(x)
        return x


model = TextClassifier()

Using cache found in /Users/obov/.cache/torch/hub/huggingface_transformers_main


In [11]:
count_of_params = 0
for param in model.encoder.parameters():
    count_of_params += 1
    param.requires_grad = False
count_of_params

146

In [12]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to("mps")
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

print_first_input = True

for epoch in range(n_epochs):
    total_loss = 0.0
    model.train()
    for data in small_train_loader:
        model.zero_grad()
        inputs, attention_mask, labels = data
        inputs, attention_mask, labels = (
            inputs.to("mps"),
            attention_mask.to("mps"),
            labels.to("mps").float(),
        )

        preds = model(
            inputs,
            attention_mask=attention_mask,
        )
        if print_first_input:
            print("preds", preds)
            print("labels", labels)
            print("Inputs:", inputs)
            print("attention_mask:", attention_mask)
            print("shape of preds", preds.shape)
            print("shape of labels", labels.shape)
            print_first_input = False
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

preds tensor([[-2.1290e+00,  3.7523e-01,  6.4188e-03, -3.2178e-01],
        [-1.4945e+00,  5.9699e-02, -2.5798e-01,  2.0558e-02],
        [-1.3085e+00,  2.6177e-02, -8.1537e-02, -1.2774e-03],
        [-1.7172e+00,  3.8724e-01, -4.8355e-01,  9.7405e-02],
        [-2.4325e+00,  2.6857e-01,  3.2639e-01,  1.1146e-01],
        [-2.1658e+00,  9.7670e-01,  1.2180e-01,  2.6788e-01],
        [-1.7178e+00,  6.4171e-01,  1.5899e-01,  2.0415e-01],
        [-2.2632e+00,  5.6679e-02, -2.3001e-02,  1.6168e-01],
        [-1.5660e+00,  3.0968e-01, -3.2449e-01,  2.6109e-01],
        [-1.4941e+00,  1.4896e-01,  2.0574e-01,  3.1735e-01],
        [-1.6123e+00,  6.7180e-02, -2.0605e-01,  1.0662e-01],
        [-1.7781e+00,  5.1833e-01, -2.8060e-01, -5.4215e-01],
        [-1.9097e+00,  1.8553e-01,  2.2390e-01,  6.4359e-01],
        [-1.7010e+00,  4.0676e-01, -8.0380e-02,  2.6657e-01],
        [-1.9443e+00,  7.5452e-01,  1.7930e-01, -3.3891e-01],
        [-2.1827e+00, -1.1448e-01,  2.1381e-01,  4.3500e-01],
  

In [13]:
def accuracy(model, dataloader):
    cnt = 0
    acc = 0

    for data in dataloader:
        inputs, attention_mask, labels = data
        inputs, attention_mask, labels = (
            inputs.to("mps"),
            attention_mask.to("mps"),
            labels.to("mps"),
        )

        preds = model(inputs, attention_mask)
        preds = torch.argmax(preds, dim=-1)
        # preds = (preds > 0).long()[..., 0]

        cnt += labels.shape[0]
        acc += (labels == preds).sum().item()

    return acc / cnt


with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, small_train_loader)
    test_acc = accuracy(model, small_test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.810 | Test acc: 0.723


In [14]:
ds["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 7600
})

In [15]:
import random


def pick_random_number():
    return random.randint(0, 7599)

In [16]:
rnd_num = pick_random_number()